In [1]:
import yaml
from modules.agent import OpenAICommentAnalysisAgent


with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')


In [2]:
agent.comment_analyze('This desk is sturdy!')

'The comment "This desk is sturdy!" directly mentions the stability of the desk, which is a positive aspect. According to the guidelines, "sturdy" is considered a positive aspect of "stability." There are no other aspects mentioned in the comment. Therefore, the extracted aspect is: {\'Pos1\': \'Stability\'}.'

In [2]:
resp = agent.points_extract("I spent up on the desktop but the slab of wood was beautiful (and heavy). The legs and other materials appear to be all metal. Wood + metal is rare these days everything is plastic.   Have only owned it for a week but love it so far. Plan to keep for the rest of my life.")
print(resp)

```json
{
  "Material Quality": "slab of wood was beautiful and heavy",
  "Material Composition": "legs and other materials appear to be all metal",
  "Rarity": "Wood + metal is rare these days everything is plastic",
  "Initial Satisfaction": "love it so far",
  "Longevity Expectation": "Plan to keep for the rest of my life"
}
```


In [3]:
import pandas as pd


df = pd.read_csv('./docs/testset_amz_bsr_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

Index(['ASIN', '标题', '标题(翻译)', '内容', '内容(翻译)', 'VP评论', 'Vine Voice评论', '型号',
       '星级', '赞同数', '图片数量', '图片地址', '是否有视频', '视频地址', '评论链接', '评论人', '头像地址',
       '所属国家', '评论人主页', '红人计划链接', '评论时间'],
      dtype='object')
(803, 21)


# Scratchs

In [18]:
import re
import yaml
import pandas as pd
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


df = pd.read_csv('./docs/testset_stations_bs_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')

for i in tqdm(range(len(df))):
    comment = df.iloc[i]['content']
    if not isinstance(comment, str):
        continue

    points = agent.points_extract(comment)
    # print(points)
    df.loc[i, '体验点分析'] = points
    extract_points = extract_output(points)
    df.loc[i, '体验点'] = ''.join(extract_points)
    # print(extract_points[0])
    # break

df.to_csv('./results/stations_bs_reviews_points.csv', sep='\t')

Index(['Unnamed: 0', 'url', 'author', 'star', 'date', 'title', 'content',
       'location', 'images[0]', 'images[1]', 'images[2]', 'images[3]',
       'image[0]', 'image[1]', 'image[2]', 'image[3]', 'image[4]', 'images[4]',
       'images[5]', 'images[6]', 'images[7]', 'images[8]', 'images[9]',
       'images[10]', 'images[11]', 'images[12]', 'images[13]', 'images[14]',
       'images[15]', 'images[16]', 'image[5]', 'image[6]', 'image[7]',
       'image[8]', 'image[9]', 'iamge'],
      dtype='object')
(706, 36)


100%|██████████| 706/706 [19:43<00:00,  1.68s/it]
